In [1]:
%load_ext autoreload
%autoreload 2
from dimension_reduction import *

In [2]:
# takes about 3.5m to run this
root_folder = 'data/MET'
images_array = load_images_from_folders(root_folder)
images_array_grey = load_images_from_folders(root_folder, mode='L')
flattened_grey = np.array([image.flatten() for image in images_array_grey])
flattened_color = np.array([image.flatten() for image in images_array])

In [3]:
import plotly.graph_objects as go
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import kneighbors_graph
from scipy.linalg import eigh

def laplacian_eigenmap_3d_plotly(data, k, image_urls):
    # Standardize data set
    scaler = StandardScaler()
    X = scaler.fit_transform(data)

    # Create adjacency matrix and make it symmetric
    adj_directed = kneighbors_graph(X, k, mode='connectivity', include_self=True).toarray()
    W = np.maximum(adj_directed, adj_directed.T)

    # Calculate Diagonal and Laplacian Matrix
    W_sum = np.sum(W, axis=1)
    D = np.diag(W_sum)
    L = D - W

    # Calculate the bottom 3 eigenvectors
    eigenvalues, eigenvectors = eigh(L)
    sorted_indices = np.argsort(eigenvalues)
    eigenvectors = eigenvectors[:, sorted_indices]
    bottom_3_eigenvectors = eigenvectors[:,1:4]

    # Plotting
    fig = go.Figure(data=[go.Scatter3d(
        x=bottom_3_eigenvectors[:,0],
        y=bottom_3_eigenvectors[:,1],
        z=bottom_3_eigenvectors[:,2],
        mode='markers',
        marker=dict(size=5, color='blue'),
        text=image_urls,  # Assuming image_urls are directly viewable
        hoverinfo='text'
    )])

    fig.update_layout(
        title=f'Scatter Plot of Bottom 3 Eigenvectors With k={k} and Sample Size = {data.shape[0]}',
        scene=dict(
            xaxis_title='First Bottom Eigenvector',
            yaxis_title='Second Bottom Eigenvector',
            zaxis_title='Third Bottom Eigenvector'
        )
    )
    
    return fig
